In [1]:

import requests
from PIL import Image
import pytesseract
from bs4 import BeautifulSoup
import re 
from datetime import datetime, timedelta

In [2]:
cookie = 'Bifrost_SessionID=vzjhjgywlgbwn2plwsw1d15n'

In [4]:
request_first_page("01/10/2024","31/12/2024")

Request 01/10/2024,31/12/2024 thành công!


'\r\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\r\n<html xmlns="http://www.w3.org/1999/xhtml">\r\n<head id="ctl00_Head1"><title>\r\n\tTìm kiếm hồ sơ C/O - Doanh nghiệp\r\n</title><meta http-equiv="Content-Type" content="text/html; charset=utf-8" /><meta name="Description" content="Bifröst, framework, system, vietnam, vietnamese, quản lý, hành chính, hành chính công, xuất khẩu, điện tử, tự động" /><meta name="Keywords" content="Bifröst, framework, system, vietnam, vietnamese, quản lý, hành chính, hành chính công, xuất khẩu, điện tử, tự động" /><meta name="Abstract" content="Bifröst, framework, system, vietnam, vietnamese, quản lý, hành chính, hành chính công, xuất khẩu, điện tử, tự động" /><meta name="google-site-verification" /><meta name="Robots" content="index,follow,all" /><meta name="Author" content="MOIT" /><meta name="Copyright" content="Copyright © 2019 MOIT. All rights reserved." /><meta name="Manage

In [3]:
def request_first_page(fromdate,todate):
    # URL yêu cầu
    url = 'https://ecosys.gov.vn/CertificatesUpgrade/Business/CertificateAdvanceSearch.aspx'

    # Đặt các tham số URL (query parameters)
    params = {
        'CertificateDateFrom': fromdate,
        'CertificateDateTo': todate,
        'Status': '-1',
        'FormCOId': '0',
        'CountryId': '0',
        'CertificateNumber': '',
        'CustomsNumber': '',
        'ReceiverName': '',
        'TransportMethodId': '0'
    }

    # Đặt headers
    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'Accept-Language': 'en-US,en;q=0.9,vi;q=0.8,vi-VN;q=0.7',
        'Cache-Control': 'max-age=0',
        'Connection': 'keep-alive',
        'Cookie': cookie,
        'DNT': '1',
        'Referer': 'https://ecosys.gov.vn/CertificatesUpgrade/Business/CertificateAdvanceSearch.aspx?',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-User': '?1',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
        'sec-ch-ua': '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"'
    }

    # Gửi yêu cầu GET
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        print(f"Request {fromdate},{todate} thành công!")
        return response.text
    # print(response2.text)  # Nội dung HTML
    else:
        print(f"Request thất bại: {response.status_code}")
        return None
